In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://docs.quantum-computing.ibm.com/run/primitives
# result = Sampler().run(circuits).result()
from qiskit.circuit import ClassicalRegister, QuantumRegister
from qiskit.circuit import QuantumCircuit
from qiskit.primitives import Estimator, Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_histogram
from qiskit.circuit import Parameter

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background') # optional
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, Aer, execute

ModuleNotFoundError: No module named 'ibm_quantum_widgets'

Making the Oracle of the Deutsch Jozsa Algorithm

In [2]:
def oracle(q_c):
    n = q_c.num_qubits
    for i in range(n-1):
        q_c.cx(i,n-1)
    return q_c    
    

Defining adaptive rotation Algorithm

In [3]:
def adaptive_rotation(theta_max,theta_min,i,iter_max):
    theta_i = theta_max-((theta_max-theta_min)/iter_max)*i
    return theta_i

Recombination Operator 

In [6]:
def recombination(m,theta_max,theta_min,t,t_max):
    qr = QuantumRegister(m+1, 'q')
    cr = ClassicalRegister(m,'q_m')
    qc = QuantumCircuit(qr,cr)
   
    qc.x(m)
    qc.barrier()
    for i in range(m+1):
        qc.h(i)
    qc.barrier()    
    oracle(qc)
    qc.barrier()  
    for i in range(m):
        qc.h(i)
    qc.barrier()     
    theta = adaptive_rotation(theta_max,theta_min,t,t_max)
    for i in range(m+1):
        qc.ry(theta,i)
    qc.barrier()    
    qc.measure(range(m), range(m))
    return qc

qc_1 = recombination(3, np.pi, -np.pi,1,100)

In [22]:
def population(qc):
    sim = Aer.get_backend('aer_simulator')
    shots = 10000
    counts_qc = execute(qc, sim, shots=shots).result().get_counts()
    plot_histogram(counts_qc, figsize=(9,5))
    # print(counts_qc)
    return list(counts_qc.keys())

print(population(recombination(3, np.pi, -np.pi,1,100)))    

['001', '100', '010', '000']


In [ ]:
binary_array = np.fromiter(binary_string, dtype=int)

In [18]:
def fitness(x,p):
    f = []
    for i in x:
        x_i = np.fromiter(i, dtype=int)
        f.append(x_i.dot(p))    
    return np.array(f)

In [19]:
#Testing
x = population(qc_1)
p = [3,1,2]
print(x)
print(fitness(x,p))


['001', '100', '010', '000']
[2 3 1 0]


In [ ]:
def linear_penalty_fitness(x,p,w,C,lamda):
    fitness_initial = fitness(x,p)
    fitness_pen = []
    for i in range(len(x)):
        x_i = np.fromiter(i, dtype = int)
        w_i = x_i.dot(w)
        if w_i>C:
            f_p = fitness_initial[i]-lamda*(w_i-C)
        else:
            f_p = fitness_initial[i]
        fitness_pen.append(f_p)
    
    return fitness_pen
    

In [23]:
def best_solution(x,fit):
    index_max = np.argmax(fit)
    b_s = x[index_max]
    return b_s

In [24]:
print(best_solution(x,fitness(x,p)))

100


In [25]:
def Q_Genetic_Algorithm(p,w,C,theta_max,theta_min,t_max):
    t = 0
    best_sol = []
    m = len(p)       #Number of items in the knapsack
    Q_t = recombination(m,theta_max,theta_min,t,t_max) #Initial Population
    for i in range(1,t_max):
        pop = population(Q_t)  #Population of generation t
        fit  = fitness(pop,p)  #List of Fitness values of chromosomes of the population
        b_sol = best_solution(pop,fit) #Finding best solution
        best_sol.append(b_sol)
        t = t+1
        Q_t = recombination(m,theta_max,theta_min,t,t_max)
        
    return best_sol


In [26]:
p = [3,1,2]
w = [1,2,3]
C = 4
theta_max = np.pi
theta_min = -np.pi
t_max = 100
print(Q_Genetic_Algorithm(p,w,C,theta_max,theta_min,t_max))

['100', '100', '101', '110', '100', '100', '101', '100', '100', '100', '100', '100', '100', '101', '100', '100', '100', '011', '100', '100', '100', '100', '100', '100', '100', '110', '011', '100', '101', '100', '101', '100', '100', '100', '110', '100', '110', '110', '011', '100', '101', '100', '110', '110', '100', '101', '110', '100', '100', '110', '100', '100', '100', '101', '110', '100', '100', '110', '101', '100', '110', '100', '100', '101', '100', '101', '110', '100', '100', '100', '100', '110', '011', '101', '100', '011', '100', '110', '100', '100', '100', '011', '100', '011', '100', '100', '011', '100', '110', '110', '100', '100', '100', '100', '011', '110', '101', '100', '101']
